In [1]:
# The molecular volume of methanol is 
Vm_cm3_per_mol = 40.5 # cm^3/mol
nm3_per_cm3 = (1.0e7)**3
Vm_nm3_per_mol = Vm_cm3_per_mol*nm3_per_cm3

# Avogadros number
N_A = 6.022e23  # (particles)  per mol = mol^{-1}

nparticles = 320
nparticles_in_moles = nparticles/N_A  # mol
V_in_nm3 = Vm_nm3_per_mol*nparticles_in_moles

box_length_in_nm = (V_in_nm3**(1/3))
print(f'box_length_in_nm for {nparticles} molecules of methanol: {box_length_in_nm}') 


box_length_in_nm for 320 molecules of methanol: 2.781557765177617


In [8]:
import gromacs
import os

In [9]:
os.system('gmx editconf -f meoh_GMX.gro -box 2.8 2.8 2.8 -o meoh_single.gro')

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 6 atoms
Volume: 62.0169 nm^3, corresponds to roughly 27900 electrons
No velocities found
    system size :  0.157  0.223  0.222 (nm)
    center      :  0.088  0.017 -0.029 (nm)
    box vectors :  3.136  4.446  4.448 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  62.02               (nm^3)
    shift       :  1.312  1.383  1.429 (nm)
new center      :  1.400  1.400  1.400 (nm)
new box vectors :  2.800  2.800  2.800 (nm)
new box angles  :  90.00  90.00  90.00 (degrees)
new box volume  :  21.95               (nm^3)


                     :-) GROMACS - gmx editconf, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
    

0

In [10]:
os.system('gmx insert-molecules -f meoh_single.gro -ci meoh_GMX.gro -box 2.8 2.8 2.8 -nmol 319 -try 20000 -o meoh_to_be_equilibrated.gro')


         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary.

NOTE: From version 5.0 gmx insert-molecules uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------



                 :-) GROMACS - gmx insert-molecules, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:


0

In [11]:
import sys
sys.path.append('../../')
from mdp_helpers import *

workdir = './work'
if not os.path.exists(workdir):
    os.mkdir(workdir)

    
import gromacs

In [12]:
## Minimize the system
em_mdpfile = f'{workdir}/em.mdp'
write_gromacs_min_mdp(em_mdpfile)

em_tprfile = f'{workdir}/em.tpr'
em_grofile = f'{workdir}/em.gro'
em_logfile = f'{workdir}/em.log'
em_trrfile = f'{workdir}/em.trr'
em_edrfile = f'{workdir}/em.edr'

input_grofile = 'meoh_to_be_equilibrated.gro'
input_topfile = 'meoh_GMX.top'

In [14]:
import subprocess

# Run GROMACS grompp command using subprocess
subprocess.run(['gmx', 'grompp', '-f', em_mdpfile, '-c', input_grofile, '-p', input_topfile, '-o', em_tprfile])

# Run GROMACS mdrun command using subprocess
subprocess.run(['gmx', 'mdrun', '-v', '-s', em_tprfile, '-o', em_trrfile, '-c', em_grofile, '-e', em_edrfile, '-g', em_logfile])

                      :-) GROMACS - gmx grompp, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

Analysing residue names:
There are:   320      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.140 0.140 0.140
This run will generate roughly 1 Mb of data



Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =       100000
Step=    0, Dmax= 1.0e-02 nm, Epot=  7.20672e+04 Fmax= 1.95654e+05, atom= 1574
Step=    1, Dmax= 1.0e-02 nm, Epot=  5.65933e+04 Fmax= 6.74994e+04, atom= 1046
Step=    2, Dmax= 1.2e-02 nm, Epot=  3.96145e+04 Fmax= 2.43626e+04, atom= 1184
Step=    3, Dmax= 1.4e-02 nm, Epot=  2.70929e+04 Fmax= 8.01542e+03, atom= 124
Step=    4, Dmax= 1.7e-02 nm, Epot=  1.90695e+04 Fmax= 3.63999e+03, atom= 899
Step=    5, Dmax= 2.1e-02 nm, Epot=  1.63754e+04 Fmax= 1.03607e+04, atom= 1388
Step=    6, Dmax= 2.5e-02 nm, Epot=  1.48455e+04 Fmax= 1.03992e+04, atom= 1405
Step=    8, Dmax= 1.5e-02 nm, Epot=  1.20811e+04 Fmax= 3.50563e+03, atom= 1405
Step=    9, Dmax= 1.8e-02 nm, Epot=  1.12397e+04 Fmax= 1.49513e+04, atom= 1405
Step=   10, Dmax= 2.1e-02 nm, Epot=  1.00948e+04 Fmax= 5.09523e+03, atom= 1405
Step=   11, Dmax= 2.6e-02 nm, Epot=  9.54505e+03 Fmax= 2.09262e+04, atom= 1405
Step=   12, Dmax= 3.1e-02 nm, Epot=  8.

CompletedProcess(args=['gmx', 'mdrun', '-v', '-s', './work/em.tpr', '-o', './work/em.trr', '-c', './work/em.gro', '-e', './work/em.edr', '-g', './work/em.log'], returncode=0)

In [15]:
# NvT equilibrate the system

def write_gromacs_nvt_equil_mdp_for_methanol(filename):
    ''' writes a gromacs mdp file for equilibration'''
    with open(filename, 'w') as f:
        f.write(f'''define                 = -DPOSRE          ; position restrain the protein
; Run parameters
integrator             = md                          ; leap-frog integrator
nsteps                 = 100000                      ; 2 * 100000 = 200 ps
dt                     = 0.002                       ; 2 fs
; Output control
nstxout                = 1000                        ; save coordinates every 2 ps
nstvout                = 1000                        ; save velocities every 2 ps
nstenergy              = 1000                        ; save energies every 2 ps
nstlog                 = 1000                        ; update log file every 2 ps
; Bond parameters
continuation           = no                          ; Initial simulation
constraint_algorithm   = lincs                       ; holonomic constraints
constraints            = h-bonds                     ; all bonds (even heavy atom-H bonds) constrained
lincs_iter             = 1                           ; accuracy of LINCS
lincs_order            = 4                           ; also related to accuracy
; Neighborsearching
ns_type                = grid                        ; search neighboring grid cels
nstlist                = 10                          ; 20 fs
rlist                  = 1.0                         ; short-range neighborlist cutoff (in nm)
rcoulomb               = 1.0                         ; short-range electrostatic cutoff (in nm)
rvdw                   = 1.0                         ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype            = PME                         ; Particle Mesh Ewald for long-range electrostatics
pme_order              = 4                           ; cubic interpolation
fourierspacing         = 0.16                        ; grid spacing for FFT
; Temperature coupling is on
tcoupl                 = V-rescale                   ; Weak coupling for initial equilibration
tc-grps                = System                      ; two coupling groups - more accurate
tau_t                  = 0.1                         ; time constant, in ps
ref_t                  = 298.15                      ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                 = Berendsen                   ; Pressure coupling on in NPT, also weak coupling
pcoupltype             = isotropic                   ; uniform scaling of x-y-z box vectors
tau_p                  = 2.0                         ; time constant, in ps
ref_p                  = 1.0                         ; reference pressure (in bar)
compressibility        = 4.5e-5                      ; isothermal compressibility, bar^-1
refcoord_scaling       = com
; Periodic boundary conditions
pbc                    = xyz                         ; 3-D PBC
; Dispersion correction
DispCorr               = EnerPres                    ; account for cut-off vdW scheme
; Velocity generationd
gen_vel                = yes                         ; Velocity generation is on
gen_temp               = 298.15                      ; temperature for velocity generation
gen_seed               = -1                          ; random seed
; These options remove COM motion of the system
nstcomm                = 10
comm-mode              = Linear
comm-grps              = System''')
    f.close()

In [16]:
## Equilibrate the system
nvt_mdpfile = f'{workdir}/nvt.mdp'
write_gromacs_nvt_equil_mdp_for_methanol(nvt_mdpfile)

nvt_tprfile = f'{workdir}/nvt.tpr'
nvt_grofile = f'{workdir}/nvt.gro'
nvt_logfile = f'{workdir}/nvt.log'
nvt_xtcfile = f'{workdir}/nvt.xtc'
nvt_trrfile = f'{workdir}/nvt.trr'
nvt_edrfile = f'{workdir}/nvt.edr'
nvt_cptfile = f'{workdir}/nvt.cpt'

In [17]:
# Run GROMACS grompp command using subprocess
subprocess.run(['gmx', 'grompp', '-f', nvt_mdpfile, '-c', em_grofile, '-p', input_topfile, '-o', nvt_tprfile, '-maxwarn', '1'])

# Run GROMACS mdrun command using subprocess
subprocess.run(['gmx', 'mdrun', '-v', '-s', nvt_tprfile, '-o', nvt_trrfile, '-x', nvt_xtcfile, '-c', nvt_grofile, '-e', nvt_edrfile, '-g', nvt_logfile, '-cpo', nvt_cptfile])

                      :-) GROMACS - gmx grompp, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

turning H bonds into constraints...
Analysing residue names:
There are:   320      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298.15 K
Calculated rlist for 1x1 atom pair-list as 1.009 nm, buffer size 0.009 nm
Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.140 0.140 0.140
This run will generate roughly 5 Mb of data


starting mdrun 'smiles_molecule'
100000 steps,    200.0 ps.
step 99900, remaining wall clock time:     0 s          imb F  2% imb F  6% l 0.92  imb F 10% vol 0.94  imb F  6% vol 0.92  imb F 10% vol 0.91  imb F  7% vol 0.91  imb F 16% vol 0.92  imb F  4% vol 0.94  imb F  8% vol 0.92  imb F  9% vol 0.93  imb F 11% vol 0.94  imb F 10% vol 0.94  imb F 11% imb F 16% imb F 10% imb F 11% imb F  4% imb F 14% imb F  6% imb F  4% imb F 13% imb F  8% imb F  8% imb F  8% imb F  9% imb F  9% imb F  5% imb F  5% imb F 16% imb F  7% imb F  5% imb F 17% imb F  6% imb F  5% imb F 12% imb F 13% imb F  4% imb F  3% imb F  7% imb F  6% imb F 14% imb F  5% imb F  5% imb F  3% imb F  4% imb F  8% imb F  6% imb F 15% imb F  3% imb F  5% imb F  8% imb F  8% imb F 13% imb F  9% imb F  7% imb F  7% imb F 10% imb F 14% imb F  8% imb F  3% imb F 15% imb F  5% imb F 18% imb F  4% imb F  5% imb F  3% imb F  6% imb F  8% imb F  3% imb F 11% imb F  9% imb F  9% imb F  6% imb F  8% imb F 10% imb F  3% imb F  7% imb F 

CompletedProcess(args=['gmx', 'mdrun', '-v', '-s', './work/nvt.tpr', '-o', './work/nvt.trr', '-x', './work/nvt.xtc', '-c', './work/nvt.gro', '-e', './work/nvt.edr', '-g', './work/nvt.log', '-cpo', './work/nvt.cpt'], returncode=0)

In [18]:
def write_gromacs_npt_equil_mdp_for_methanol(filename):
    ''' writes a gromacs mdp file for equilibration'''
    with open(filename, 'w') as f:
        f.write(f'''define                 = -DPOSRE          ; position restrain the protein
; Run parameters
integrator             = md                          ; leap-frog integrator
nsteps                 = 100000                      ; 2 * 100000 = 200 ps
dt                     = 0.002                       ; 2 fs
; Output control
nstxout                = 1000                        ; save coordinates every 2 ps
nstvout                = 1000                        ; save velocities every 2 ps
nstenergy              = 1000                        ; save energies every 2 ps
nstlog                 = 1000                        ; update log file every 2 ps
; Bond parameters
continuation           = no                          ; Initial simulation
constraint_algorithm   = lincs                       ; holonomic constraints
constraints            = h-bonds                     ; all bonds (even heavy atom-H bonds) constrained
lincs_iter             = 1                           ; accuracy of LINCS
lincs_order            = 4                           ; also related to accuracy
; Neighborsearching
ns_type                = grid                        ; search neighboring grid cels
nstlist                = 10                          ; 20 fs
rlist                  = 1.0                         ; short-range neighborlist cutoff (in nm)
rcoulomb               = 1.0                         ; short-range electrostatic cutoff (in nm)
rvdw                   = 1.0                         ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype            = PME                         ; Particle Mesh Ewald for long-range electrostatics
pme_order              = 4                           ; cubic interpolation
fourierspacing         = 0.16                        ; grid spacing for FFT
; Temperature coupling is on
tcoupl                 = V-rescale                   ; Weak coupling for initial equilibration
tc-grps                = System                      ; two coupling groups - more accurate
tau_t                  = 0.1                         ; time constant, in ps
ref_t                  = 298.15                      ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                 = Berendsen                   ; Pressure coupling on in NPT, also weak coupling
pcoupltype             = isotropic                   ; uniform scaling of x-y-z box vectors
tau_p                  = 2.0                         ; time constant, in ps
ref_p                  = 1.0                         ; reference pressure (in bar)
compressibility        = 4.5e-5                      ; isothermal compressibility, bar^-1
refcoord_scaling       = com
; Periodic boundary conditions
pbc                    = xyz                         ; 3-D PBC
; Dispersion correction
DispCorr               = EnerPres                    ; account for cut-off vdW scheme
; Velocity generationd
gen_vel                = yes                         ; Velocity generation is on
gen_temp               = 298.15                      ; temperature for velocity generation
gen_seed               = -1                          ; random seed
; These options remove COM motion of the system
nstcomm                = 10
comm-mode              = Linear
comm-grps              = System''')
    f.close()

In [19]:
## Equilibrate the system
workdir = './work'
npt_mdpfile = f'{workdir}/npt.mdp'
write_gromacs_npt_equil_mdp_for_methanol(npt_mdpfile)

nvt_tprfile = f'{workdir}/nvt.tpr'
nvt_grofile = f'{workdir}/nvt.gro'
nvt_logfile = f'{workdir}/nvt.log'
nvt_xtcfile = f'{workdir}/nvt.xtc'
nvt_trrfile = f'{workdir}/nvt.trr'
nvt_edrfile = f'{workdir}/nvt.edr'
nvt_cptfile = f'{workdir}/nvt.cpt'
input_topfile = 'meoh_GMX.top'

npt_tprfile = f'{workdir}/npt.tpr'
npt_grofile = f'{workdir}/npt.gro'
npt_logfile = f'{workdir}/npt.log'
npt_xtcfile = f'{workdir}/npt.xtc'
npt_trrfile = f'{workdir}/npt.trr'
npt_edrfile = f'{workdir}/npt.edr'
npt_cptfile = f'{workdir}/npt.cpt'


In [20]:
subprocess.run(['gmx', 'grompp', '-f', npt_mdpfile, '-c', nvt_grofile, '-p', input_topfile, '-o', npt_tprfile, '-maxwarn', '1'])
subprocess.run(['gmx', 'mdrun', '-v', '-s', npt_tprfile, '-o', npt_trrfile, '-x', npt_xtcfile, '-c', npt_grofile, '-e', npt_edrfile, '-g', npt_logfile, '-cpo', npt_cptfile])


                      :-) GROMACS - gmx grompp, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

turning H bonds into constraints...
Analysing residue names:
There are:   320      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 298.15 K
Calculated rlist for 1x1 atom pair-list as 1.009 nm, buffer size 0.009 nm
Set rlist, assuming 4x4 atom pair-list, to 1.000 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 20x20x20, spacing 0.139 0.139 0.139
This run will generate roughly 5 Mb of data


starting mdrun 'smiles_molecule'
100000 steps,    200.0 ps.
step 99900, remaining wall clock time:     0 s          imb F  5% imb F  6% b F 13% vol 0.95  imb F  6% vol 0.93  imb F  9% vol 0.92  imb F 10% vol 0.92  imb F  8% vol 0.94  imb F  4% vol 0.93  imb F 16% vol 0.93  imb F 23% vol 0.93  imb F  8% imb F  5% imb F  7% imb F  9% imb F  9% imb F 11% imb F  7% imb F  7% imb F  6% imb F  6% imb F  8% imb F  7% imb F  3% imb F  5% imb F  4% imb F  4% imb F  4% imb F  7% imb F  9% imb F  4% imb F 13% imb F  9% imb F  9% imb F  7% imb F  8% imb F  5% imb F  6% imb F  6% imb F  7% imb F  7% vol 0.86  imb F 12% vol 0.95  imb F  5% vol 0.95  imb F  4% vol 0.96  imb F 10% vol 0.96  imb F  7% vol 0.97  imb F  5% vol 0.92  imb F  6% vol 0.90  imb F  6% vol 0.95  imb F  6% vol 0.94  imb F  4% vol 0.95  imb F  5% vol 0.93  imb F  4% vol 0.91  imb F  6% vol 0.90  imb F  9% vol 0.93  imb F  4% vol 0.95  imb F  4% vol 0.94  imb F  3% vol 0.95  imb F  4% vol 0.92  imb F  6% vol 0.91  imb F  6% vol 0.

CompletedProcess(args=['gmx', 'mdrun', '-v', '-s', './work/npt.tpr', '-o', './work/npt.trr', '-x', './work/npt.xtc', '-c', './work/npt.gro', '-e', './work/npt.edr', '-g', './work/npt.log', '-cpo', './work/npt.cpt'], returncode=0)

In [21]:
import shutil

# Copy the npt.gro file to the desired location
source_file = 'work/npt.gro'
destination_file = 'meoh_320_box.gro'

# Use shutil.copy to copy the file
shutil.copy(source_file, destination_file)

# Verify the file copy
if os.path.exists(destination_file):
    print(f"File copied successfully to {destination_file}")
else:
    print("File copy failed.")

File copied successfully to meoh_320_box.gro


In [22]:
import shutil

# Copy the npt.gro file to the desired location
source_file = 'smiles_molecule.acpype/smiles_molecule_GMX.itp'
destination_file = 'meoh_320_box.itp'

# Use shutil.copy to copy the file
shutil.copy(source_file, destination_file)

# Verify the file copy
if os.path.exists(destination_file):
    print(f"File copied successfully to {destination_file}")
else:
    print("File copy failed.")

File copied successfully to meoh_320_box.itp
